In [ ]:
#make corpus from nltk stuff(won't have to preprocess much)
#train glove on that corpus
#define functions you need
#what do you need bruhh
#make that nice co-occurrence matrix
#figure out how optimisation works
#figure out initiallisation
#figure out training shizz
#figure out how you exporting yo model and shizz


In [2]:
from scipy import sparse
import numpy as np
from keras.layers import Input, Embedding, Dot, Reshape, Add
from keras.models import Model
from keras.optimizers import Adam
import nltk

In [19]:
import nltk
from nltk.corpus import brown

In [20]:
nltk.download('brown')
data = brown.sents(categories=brown.categories())
len(data)

[nltk_data] Downloading package brown to /home/shiven/nltk_data...
[nltk_data]   Package brown is already up-to-date!


57340

In [50]:
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
punctuation = ['!','"','#','$','%','&',"'",'(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~','``',"''",'--']
lower_words = [x.lower() for x in brown.words()]
pun_stop = punctuation + stopwords.words('english')
filter_words1 = [x for x in lower_words if x not in pun_stop]
filter_words = list(filter(lambda x: x.isalpha() and len(x) > 1, filter_words1)) 

[nltk_data] Downloading package stopwords to /home/shiven/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
sentences=[]
stopwords_ = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

for sentence in data:
    for word in stopwords_:
        token=" "+word+" "
        sentence=[item.replace(token," ") for item in sentence]
    
    sentences.append(sentence)


57340


In [37]:
for i in range(len(sentences)):
    for pun in punctuation:
        sentences[i]=[item for item in sentences[i]]
        sentences[i]=[item for item in sentences[i] if item]

In [38]:
print(sentences[0])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlantas', 'recent', 'primary', 'election', 'produced', 'no', 'evidence', 'that', 'any', 'irregularities', 'took', 'place']


In [39]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [51]:
vocab_size=10000
vector_dim=100
maxlen=30

In [52]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [53]:
sequences = tokenizer.texts_to_sequences(sentences)
sequences_padded = pad_sequences(sequences, padding='post', maxlen=maxlen)


In [54]:
print(sentences[4000])
print(sequences[4000])
print(sequences_padded[4000])
print(len(sentences[4000]))
print(len(sequences[4000]))
print(len(sequences_padded[4000]))

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlantas', 'recent', 'primary', 'election', 'produced', 'no', 'evidence', 'that', 'any', 'irregularities', 'took', 'place']
[2, 5437, 643, 2297, 1634, 55, 1840, 30, 2180, 3, 1, 543, 1121, 1399, 1190, 50, 464, 8, 75, 9172, 203, 161]
[   2 5437  643 2297 1634   55 1840   30 2180    3    1  543 1121 1399
 1190   50  464    8   75 9172  203  161    0    0    0    0    0    0
    0    0]
22
22
30
